## Process and visualise the results of the multiobjective dam porftolio selection
### T. Janus
### Created: 09/11/2024

## TODO:
1. Visualise different dam scenarios on maps (borrow the maps from one of the previous notebooks
2. Create a composite figure with tiles using facetgrid etc.

In [ ]:
from typing import ClassVar, Dict, List, Any, Tuple, Set, Tuple, Sequence
from typing import TypeAlias, TypeVar, Generic
import subprocess
import pathlib
import numpy as np
import pandas as pd
import string
from datetime import datetime
from parse import parse
import json
import gc
import pprint
import re
import ast
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import pygmo as pg
from tqdm import tqdm
import seaborn as sns # for Data visualization
import matplotlib
import matplotlib.pyplot as plt # for Data visualization
from IPython.display import display, HTML
from jinja2 import Template
from lib.notebook12 import (
    reduce_mem_usage, read_id_ifc_map, set_remap, get_every_n_row, 
    SolutionFileParser, OutputVisualiser, ObjectiveCalculator, 
    map_ids, find_solution_by_dam_numbers, return_row_by_criterion
)
%matplotlib inline

In [ ]:
# Paths to output files from the algorithm with expansion / compression
sol_file_folder_5obj = pathlib.Path('moo_solver_CPAIOR/outputs/epsilon2_5obj')
sol_file_folder_3obj = pathlib.Path('moo_solver_CPAIOR/outputs/epsilon2_3obj_soued')
exec_options = {
    '5obj' : {
        'nobj': 5,
        'merged_csv_file': sol_file_folder_5obj / pathlib.Path('merged_df_5obj.csv'),
        'nondom_csv_file': sol_file_folder_5obj / pathlib.Path('em_int_nondom_df_5obj.csv'),
        'max_y_int' : 70,
        'max_y_em': 5,
        'foregone_arrow_top': 0.95,
        'tot_em_txt_offset': 2.5,
        'em_int_txt_offset': 34
    },
    '5obj_soued': {
        'nobj': 5,
        'merged_csv_file': sol_file_folder_5obj / pathlib.Path('merged_df_5obj_soued.csv'),
        'nondom_csv_file': sol_file_folder_5obj / pathlib.Path('em_int_nondom_df_5obj_soued.csv'),
        'max_y_int' : 130,
        'max_y_em': 8,
        'foregone_arrow_top': 2,
        'tot_em_txt_offset': 4,
        'em_int_txt_offset': 65
    },
    '3obj_soued' : {
        'nobj': 3,
        'merged_csv_file': sol_file_folder_3obj / pathlib.Path('merged_df_3obj_soued.csv'),
        'nondom_csv_file': sol_file_folder_3obj / pathlib.Path('em_int_nondom_df_3obj_soued.csv'),
        'max_y_int' : 130,
        'max_y_em': 8,
        'foregone_arrow_top': 2,
        'tot_em_txt_offset': 4,
        'em_int_txt_offset': 65
    }
}

In [ ]:
option = '5obj'

In [ ]:
merged_df = pd.read_csv(exec_options[option]['merged_csv_file'])
em_int_nondom_df = pd.read_csv(exec_options[option]['nondom_csv_file'])
merged_df.columns

In [ ]:
# Load dataframe with dam ids and objective values for each dam
dam_data_filename = pathlib.Path("outputs/moo/all_hp.csv")
dam_df = pd.read_csv(dam_data_filename, index_col=0).set_index('ifc_id')
# Load the mapping between ids used in the MOO algorithm and the IDs in the IFC database
map_file_path = pathlib.Path('outputs/moo/id_to_ifc.json')
# Some repetition here, but left for now in fear of breaking the code
with open(map_file_path, 'r') as file:
    id_map = json.load(file)
id_map: Dict[int, int] = {int(key): value for key, value in id_map.copy().items()} # Maps optim ids to ifc ids

In [ ]:
# Find a solution with built dams with minumum energy generation (solution with built dams only)
built_min = merged_df[merged_df['Scenario']=='Built'].iloc[0]
built_max = merged_df[merged_df['Scenario']=='Built'].iloc[-1]
built_min_damids = built_min['Dam IDs']
built_min_energy = built_min["Mean annual HP, [MW]"]
built_max_energy = built_max["Mean annual HP, [MW]"]
built_min_ghg_intensity = built_min['GHG intensity [gCO<sub>2,eq</sub>/kWh]']
built_max_ghg_intensity = built_max['GHG intensity [gCO<sub>2,eq</sub>/kWh]']
oc_built = ObjectiveCalculator(dam_df, ids=set_remap(built_min['Dam IDs'], id_map))
print(
    f"Min HP from optimization: {built_min_energy} MW, min HP from input data: {oc_built.objectives['HP_mean']} MW")

In [ ]:
# Find total energy from input data using selected dams as an input
# Load dataframe with dam ids and objective values for each dam
dam_data_filename = pathlib.Path("outputs/moo/all_hp.csv")
# Load the mapping between ids used in the MOO algorithm and the IDs in the IFC database
map_file_path = pathlib.Path('outputs/moo/id_to_ifc.json')
dam_df = pd.read_csv(dam_data_filename, index_col=0).set_index('ifc_id')
with open(map_file_path, 'r') as file:
    id_map = json.load(file)
id_map: Dict[int, int] = {int(key): value for key, value in id_map.copy().items()} # Maps optim ids to ifc ids
oc_built = ObjectiveCalculator(dam_df, ids=set_remap(built_min['Dam IDs'], id_map))
print(
    f"Min HP from optimization: {built_min_energy} MW, min HP from input data: {oc_built.objectives['HP_mean']} MW")

In [ ]:
# Check retrieval of solutions from the dataframe (not used)
dam_ids_7848 = merged_df.loc[7848, 'Dam IDs']
dam_ids_7599 = merged_df.loc[7599, 'Dam IDs']

In [ ]:
em_int_nondom_df.head()

In [ ]:
merged_df.head(2)

In [ ]:
# Get selected data points
# Separate DAM IDs and No. of Dams from em_int_nondom_df and later join with plot data
#plot_data = em_int_nondom_df.drop(columns=['Dam IDs'])
#dam_ids = em_int_nondom_df[['Scenario','Dam IDs']]
em_int_nondom_df['HP Production [TWh/year]'] = em_int_nondom_df['HP Production [GWh/year]'] / 1_000
plot_data = em_int_nondom_df.apply(pd.to_numeric, errors='ignore')
#

current_solution = (built_min['HP Production [GWh/year]']/1_000, built_min_ghg_intensity)
notbuilt_current = return_row_by_criterion(
    plot_data[plot_data['Scenario']=="Not Built"], 
    'HP Production [TWh/year]', value=built_min['HP Production [GWh/year]']/1_000)

# Get selected data points
current_solution = (built_min['HP Production [GWh/year]']/1_000, built_min_ghg_intensity)
max_solution = (built_max['HP Production [GWh/year]']/1_000, built_max_ghg_intensity)
notbuilt_current = return_row_by_criterion(
    plot_data[plot_data['Scenario'] == "Not Built"], 'HP Production [TWh/year]', 
    value=built_min['HP Production [GWh/year]']/1_000)
# Find points (soluions) corresponding to HP production of 100 and 200 TWh/year respectively.
built_100 = return_row_by_criterion(
    plot_data[plot_data['Scenario'] == "Built"], 'HP Production [TWh/year]', value=100)
notbuilt_100 = return_row_by_criterion(
    plot_data[plot_data['Scenario'] == "Not Built"], 'HP Production [TWh/year]', value=100)
built_200 = return_row_by_criterion(
    plot_data[plot_data['Scenario'] == "Built"], 'HP Production [TWh/year]', value=200)
notbuilt_200 = return_row_by_criterion(
    plot_data[plot_data['Scenario'] == "Not Built"], 'HP Production [TWh/year]', value=200)

In [ ]:
# Define bin edges for firm power ratio
firm_power_bins2 = [0, 15, 30, 45, 60, 75]
firm_power_labels2 = ['0%-15%', '15%-30%', '30%-45%', '45%-60%', '60%-75%']
em_int_nondom_df['Firm Power Ratio Cat'] = pd.cut(
    em_int_nondom_df['Firm Power Ratio, [%]'], bins=firm_power_bins2, 
    labels=firm_power_labels2, right=False)
merged_df['Firm Power Ratio Cat'] = pd.cut(
    merged_df['Firm Power Ratio, [%]'], bins=firm_power_bins2, 
    labels=firm_power_labels2, right=False)
# Order categorical variables
category_order = ['0-300 km2', '300-500 km2', '500-1000 km2', '1000-1500 km2', '1500-2000 km2']
em_int_nondom_df['Loss of Land [km<sup>2</sup>]'] = \
    pd.Categorical(em_int_nondom_df['Loss of Land [km<sup>2</sup>]'], categories=category_order, ordered=True)

In [ ]:
def get_x_y_intensity_plot(data: pd.Series) -> Tuple[float, float]:
    """Return a tuple consisting of:
        (a) annual HP generation and (b) GHG intensity"""
    return (data['HP Production [TWh/year]'], data['GHG intensity [gCO<sub>2,eq</sub>/kWh]'])

def get_x_y_emissions_plot(data: pd.Series) -> Tuple[float, float]:
    """Return a tuple consisting of:
        (a) annual HP generation and (b) total GHG emissions"""
    return (
        data['HP Production [TWh/year]'], 
        data['GHG emissions [tonne CO<sub>2,eq</sub>/year]'] / 1_000_000)

In [ ]:
def make_ghg_intensity_figure(
        ax: matplotlib.axes.Axes,
        num_objectives: int,
        max_y: float,
        annotation_font_size: int = 17, 
        cur_sol_marker_size: int = 20,
        tick_label_size: int = 15,
        label_font_size: int = 15,
        sol_marker_size: int = 14) -> None:
    """ """
    marker_edge_width = 0.7
    marker_alpha = 0.9
    bbox = dict(boxstyle="round", pad=0.15, facecolor='none', edgecolor='none')
    annotation_shrink = 0.03

    kwargs  =   {
        'edgecolor':'grey',
        'marker': 'o',
        'facecolor':'none',
        'linewidth':0.05,
        'linestyle':'-',
        'alpha': 0.05
    }
    
    kwargs_ghg_intensity  =   {
        'edgecolor':'k',
        #'marker': 'o',
        'alpha': 0.05,
        'linewidth':0.05,
        'linestyle':'-',
    }
    custom_palette = ["#FFFFFF", "#808080", "#000000", "#3A0CA3", "#4361EE", "#4CC9F0"]
    merged_df['HP Production [TWh/year]'] = merged_df['HP Production [GWh/year]'] / 1_000
    
    sns.scatterplot(
        x = 'HP Production [TWh/year]', 
        y = 'GHG intensity [gCO<sub>2,eq</sub>/kWh]', 
        ax = ax,
        data = merged_df, 
        #palette='Set2', #'YlOrRd', 
        #hue='Firm Power Ratio Cat',
        s = 50,
        **kwargs)
    
    sns.set_style('white')
    sns.set_context("paper", font_scale = 1)
    sns.despine(right = True)
    if num_objectives == 3:
        sns.scatterplot(
            x = 'HP Production [TWh/year]', 
            y = 'GHG intensity [gCO<sub>2,eq</sub>/kWh]', 
            data = em_int_nondom_df, palette='Set2',
            ax =ax,
            size='Firm Power Ratio Cat',
            sizes = {'0%-15%': 10, '15%-30%': 50, '30%-45%': 100, '45%-60%': 150, '60%-75%': 200},
            #size="Loss of Land [km<sup>2</sup>]", hue='Firm Power Ratio Cat', 
            #sizes = {'0-400 km2': 10, '400-800 km2': 30, '800-1200 km2': 60, '1200-1600 km2': 120},
            **kwargs_ghg_intensity)
    else:
        sns.scatterplot(
            x = 'HP Production [TWh/year]', y = 'GHG intensity [gCO<sub>2,eq</sub>/kWh]', 
            data = em_int_nondom_df, palette='Set2',
            ax =ax,
            size='Firm Power Ratio Cat', hue="Loss of Land [km<sup>2</sup>]",
            sizes = {'0%-15%': 10, '15%-30%': 50, '30%-45%': 100, '45%-60%': 150, '60%-75%': 200},
            #size="Loss of Land [km<sup>2</sup>]", hue='Firm Power Ratio Cat', 
            #sizes = {'0-400 km2': 10, '400-800 km2': 30, '800-1200 km2': 60, '1200-1600 km2': 120},
            **kwargs_ghg_intensity)
    if num_objectives == 3:
        legend1 = ax.legend(loc="upper right", fontsize='large', ncols=1)
        legend1.set_frame_on(False)
        updated_text = [
            "Firm Power Ratio",
            "$0\% - 15\%$",
            "$15\% - 30\%$",
            "$30\% - 45\%$",
            "$45\% - 60\%$",
            "$60\% - 75\%$"
        ]
    else:
        legend1 = ax.legend(loc="upper right", fontsize='large', ncols=2)
        legend1.set_frame_on(False)
        updated_text = [
            "Loss of Land",
            "$0-300$ km$^2$",
            "$300-500$ km$^2$",
            "$500-1000$ km$^2$",
            "$1000-1500$ km$^2$",
            "$1500-2000$ km$^2$",
            "Firm Power Ratio",
            "$0\% - 15\%$",
            "$15\% - 30\%$",
            "$30\% - 45\%$",
            "$45\% - 60\%$",
            "$60\% - 75\%$"
        ]
        for ix, text in enumerate(updated_text):
            legend1.get_texts()[ix].set_text(text)
    #for ix, text in enumerate(updated_text):
    #    legend1.get_texts()[ix].set_text(text)
    ax.axvline(
        x=current_solution[0], color='grey', linestyle='--', linewidth=2,
        label='Current mean annual power production, GWh/year')
    ax.text(current_solution[0]-8, exec_options[option]['em_int_txt_offset'], 
            'Current mean annual HP production', 
             rotation=90, va='center', ha='center', fontsize=12)

    # Plot chosen solution scenarios
    ax.plot(
        current_solution[0], current_solution[1], 
        marker='*', markersize=cur_sol_marker_size, color='yellow', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.plot(
        max_solution[0], max_solution[1], 
        marker='d', markersize=cur_sol_marker_size*0.8, color="#FDB462", markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.annotate(
        '$I_{b}$', 
        xy=(current_solution[0], current_solution[1]), 
        xytext=(current_solution[0] + 20, current_solution[1] - 7.5),
        arrowprops=dict(
         facecolor='black', 
         shrink=annotation_shrink, 
         width=2, headwidth = 8 ),
        fontsize=annotation_font_size, bbox=bbox)
    # Current not built
    curr_nbuilt_x, curr_nbuilt_y = get_x_y_intensity_plot(notbuilt_current)
    ax.annotate(
        '$I_{nb}$',
        xy=(curr_nbuilt_x, curr_nbuilt_y), 
        xytext=(curr_nbuilt_x + 13, curr_nbuilt_y + 10),
        arrowprops=dict(
         facecolor='black', 
         shrink=annotation_shrink, 
         width=2, 
         headwidth = 8 ),
        fontsize=annotation_font_size, bbox=bbox)
    ax.plot(
        curr_nbuilt_x, curr_nbuilt_y, 
        marker='o', markersize=sol_marker_size, color='white', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    # Other values
    built_100_x, built_100_y = get_x_y_intensity_plot(built_100)
    ax.plot(
        built_100_x, built_100_y, 
        marker='o', markersize=sol_marker_size, color='white', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.annotate('$II_b$', xy=(built_100_x, built_100_y), xytext=(built_100_x + 20, built_100_y + 13),
                 arrowprops=dict(facecolor='black', shrink=annotation_shrink, width=2, headwidth = 8 ),
                 fontsize=annotation_font_size, bbox=bbox)
    notbuilt_100_x, notbuilt_100_y = get_x_y_intensity_plot(notbuilt_100)
    ax.plot(
        notbuilt_100_x, notbuilt_100_y, 
        marker='o', markersize=sol_marker_size, color='white', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.annotate('$II_{nb}$', xy=(notbuilt_100_x, notbuilt_100_y), xytext=(notbuilt_100_x + 38, notbuilt_100_y + 19),
                 arrowprops=dict(facecolor='black', shrink=annotation_shrink, width=2, headwidth = 8 ),
                 fontsize=annotation_font_size, bbox=bbox)
    built_200_x, built_200_y = get_x_y_intensity_plot(built_200)
    ax.plot(
        built_200_x, built_200_y, 
        marker='o', markersize=sol_marker_size, color='white', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.annotate('$III_{b}$', xy=(built_200_x, built_200_y), xytext=(built_200_x + 15, built_200_y + 9),
                 arrowprops=dict(facecolor='black', shrink=annotation_shrink, width=2, headwidth = 8 ),
                 fontsize=annotation_font_size, bbox=bbox)
    notbuilt_200_x, notbuilt_200_y = get_x_y_intensity_plot(notbuilt_200)
    ax.plot(
        notbuilt_200_x, notbuilt_200_y, 
        marker='o', markersize=sol_marker_size, color='white', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.annotate('$III_{nb}$', xy=(notbuilt_200_x, notbuilt_200_y), 
                 xytext=(notbuilt_200_x + 25, notbuilt_200_y -7),
                 arrowprops=dict(facecolor='black', shrink=annotation_shrink, width=2, headwidth = 8 ),
                 fontsize=annotation_font_size, bbox=bbox)

    ax.tick_params(axis='x', labelsize=tick_label_size)
    ax.tick_params(axis='y', labelsize=tick_label_size)
    ax.set_ylim(-5,max_y)
    ax.set_xlim(0,250)
    ax.set_xlabel("HP Production, TWh/year", fontsize=label_font_size)
    ax.set_ylabel("Emission intensity, gCO$_{2e}$/kWh", fontsize=label_font_size)
    
def make_ghg_emissions_figure(
        ax: matplotlib.axes.Axes, 
        num_objectives: int,
        max_y: float,
        annotation_font_size: int = 18, 
        tick_label_size: int = 15,
        sol_marker_size: int = 14,
        cur_sol_marker_size: int = 20,
        label_font_size: int = 15,
        vline_font_size: int = 12) -> None:
    """ """
    marker_edge_width = 0.7
    marker_alpha = 1
    bbox = dict(boxstyle="round", pad=0.15, facecolor='none', edgecolor='none')
    annotation_shrink = 0.03
    
    merged_df['HP Production [TWh/year]'] = merged_df['HP Production [GWh/year]'] / 1_000
    merged_df['GHG emissions [Mt CO2/year]'] = \
        merged_df['GHG emissions [tonne CO<sub>2,eq</sub>/year]'] / 1_000_000
    em_int_nondom_df['GHG emissions [Mt CO2/year]'] = \
        em_int_nondom_df['GHG emissions [tonne CO<sub>2,eq</sub>/year]'] / 1_000_000

    kwargs  =   {
        'edgecolor':'grey',
        'marker': 'o',
        'facecolor':'none',
        'linewidth':0.05,
        'linestyle':'-',
        'alpha': 0.05
    }
    kwargs_nondom  =   {
        'edgecolor':'k',
        'marker': 'o',
        'linewidth':0.05,
        'linestyle':'-',
        'alpha': 0.05
    }
    custom_palette = ["#FFFFFF", "#808080", "#000000", "#3A0CA3", "#4361EE", "#4CC9F0"]

    sns.set_style('white')
    sns.set_context("paper", font_scale = 1)
    sns.despine(right = True)
    # Plot all results
    
    sns.scatterplot(
        x = 'HP Production [TWh/year]', 
        y = 'GHG emissions [Mt CO2/year]', 
        ax = ax,
        data = merged_df, 
        #palette='Set2', #'YlOrRd', 
        #hue='Firm Power Ratio Cat',
        s = 50,
        **kwargs)

    #ax.legend(title='Development scenario / Firm Energy, MW', fontsize=10, 
    #          title_fontsize=12, frameon=False,
    #          ncol=3)

    if num_objectives == 3:
        # Plot nondominated results
        sns.scatterplot(
            x = 'HP Production [TWh/year]', y = 'GHG emissions [Mt CO2/year]', 
            data = em_int_nondom_df, 
            palette='Set2',
            size='Firm Power Ratio Cat',
            ax = ax,
            #sizes = {'0-400 km2': 10, '400-800 km2': 30, '800-1200 km2': 60, '1200-1600 km2': 120},
            sizes = {'0%-15%': 10, '15%-30%': 50, '30%-45%': 100, '45%-60%': 150, '60%-75%': 200},
            #sizes = (1,200),
            **kwargs_nondom)
    else:
        # Plot nondominated results
        sns.scatterplot(
            x = 'HP Production [TWh/year]', y = 'GHG emissions [Mt CO2/year]', 
            data = em_int_nondom_df, 
            palette='Set2',
            size='Firm Power Ratio Cat', hue="Loss of Land [km<sup>2</sup>]", 
            ax = ax,
            #sizes = {'0-400 km2': 10, '400-800 km2': 30, '800-1200 km2': 60, '1200-1600 km2': 120},
            sizes = {'0%-15%': 10, '15%-30%': 50, '30%-45%': 100, '45%-60%': 150, '60%-75%': 200},
            #sizes = (1,200),
            **kwargs_nondom)    

    if num_objectives == 3:
        legend2 = ax.legend(loc="upper left", fontsize='large', bbox_to_anchor=(0.08, 1.05), ncol=1)
        legend2.set_frame_on(False)
        updated_text = [
            "Firm Power Ratio",
            "$0\% - 15\%$",
            "$15\% - 30\%$",
            "$30\% - 45\%$",
            "$45\% - 60\%$",
            "$60\% - 75\%$"
        ]
    else:
        legend2 = ax.legend(loc="upper left", fontsize='large', bbox_to_anchor=(0.08, 1.05), ncol=2)
        legend2.set_frame_on(False)
        updated_text = [
            "Loss of Land",
            "$0-300$ km$^2$",
            "$300-500$ km$^2$",
            "$500-1000$ km$^2$",
            "$1000-1500$ km$^2$",
            "$1500-2000$ km$^2$",
            "Firm Power Ratio",
            "$0\% - 15\%$",
            "$15\% - 30\%$",
            "$30\% - 45\%$",
            "$45\% - 60\%$",
            "$60\% - 75\%$"
        ]        
        for ix, text in enumerate(updated_text):
            legend2.get_texts()[ix].set_text(text)
    #for ix, text in enumerate(updated_text):
    #    print(ix, len(updated_text))
    #    legend2.get_texts()[ix].set_text(text)

    # Plot solution points
    current_solution = (
        built_min['HP Production [GWh/year]']/1_000, 
        built_min['GHG emissions [tonne CO<sub>2,eq</sub>/year]']/1_000/1_000)
    ax.plot(
        current_solution[0], current_solution[1], 
        marker='*', markersize=cur_sol_marker_size, color='yellow', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.annotate('$I_{b}$', xy=(current_solution[0], current_solution[1]), 
                 xytext=(current_solution[0] + 10, current_solution[1] + 0.55),
                 arrowprops=dict(facecolor='black', shrink=annotation_shrink, width=2, headwidth = 8 ),
                 fontsize=annotation_font_size, bbox=bbox)
    
    # Current not built (point)
    curr_nbuilt_x, curr_nbuilt_y = get_x_y_emissions_plot(notbuilt_current)
    ax.plot(
        curr_nbuilt_x, curr_nbuilt_y, 
        marker='o', markersize=sol_marker_size, color='white', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.annotate('$I_{nb}$', xy=(curr_nbuilt_x, curr_nbuilt_y), xytext=(curr_nbuilt_x + 25, curr_nbuilt_y + 1.5),
                 arrowprops=dict(facecolor='black', shrink=annotation_shrink, width=2, headwidth = 8 ),
                 fontsize=annotation_font_size, bbox=bbox)
    # Other point values
    built_100_x, built_100_y = get_x_y_emissions_plot(built_100)
    ax.plot(
        built_100_x, built_100_y, 
        marker='o', markersize=sol_marker_size, color='white', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.annotate('$II_b$', xy=(built_100_x, built_100_y), xytext=(built_100_x + 11, built_100_y + 0.64),
                 arrowprops=dict(facecolor='black', shrink=annotation_shrink, width=2, headwidth = 8 ),
                 fontsize=annotation_font_size, bbox=bbox)
    notbuilt_100_x, notbuilt_100_y = get_x_y_emissions_plot(notbuilt_100)
    ax.plot(
        notbuilt_100_x, notbuilt_100_y, 
        marker='o', markersize=sol_marker_size, color='white', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.annotate('$II_{nb}$', xy=(notbuilt_100_x, notbuilt_100_y), xytext=(notbuilt_100_x + 30, notbuilt_100_y + 1.6),
                 arrowprops=dict(facecolor='black', shrink=annotation_shrink, width=2, headwidth = 8 ),
                 fontsize=annotation_font_size, bbox=bbox)
    built_200_x, built_200_y = get_x_y_emissions_plot(built_200)
    ax.plot(
        built_200_x, built_200_y, 
        marker='o', markersize=sol_marker_size, color='white', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.annotate('$III_{b}$', xy=(built_200_x, built_200_y), xytext=(built_200_x - 15, built_200_y + 0.7),
                 arrowprops=dict(facecolor='black', shrink=annotation_shrink, width=2, headwidth = 8 ),
                 fontsize=annotation_font_size, bbox=bbox)
    notbuilt_200_x, notbuilt_200_y = get_x_y_emissions_plot(notbuilt_200)
    ax.plot(
        notbuilt_200_x, notbuilt_200_y, 
        marker='o', markersize=sol_marker_size, color='white', markeredgecolor='k', 
        markeredgewidth=marker_edge_width, alpha=marker_alpha)
    ax.annotate('$III_{nb}$', xy=(notbuilt_200_x, notbuilt_200_y), 
                 xytext=(notbuilt_200_x + 26, notbuilt_200_y -0.25),
                 arrowprops=dict(facecolor='black', shrink=annotation_shrink, width=2, headwidth = 8 ),
                 fontsize=annotation_font_size, bbox=bbox) 
    
    # Add vertical line denoting the current HP production
    ax.axvline(
        x=current_solution[0], color='grey', linestyle='--', linewidth=2,
        label='Current mean annual power production, GWh/year')
    ax.text(current_solution[0]-8, exec_options[option]['tot_em_txt_offset'], 'Current mean annual HP production', 
             rotation=90, va='center', ha='center', fontsize=vline_font_size)

    arrow_x = current_solution[0] + 20
    arrow_y_start = 0  # y-coordinate for the starting point of the arrow
    arrow_y_end = exec_options[option]['foregone_arrow_top']  # y-coordinate for the ending point of the arrow
    arrow_text = 'Forgone opportunity'  # Text to be displayed next to the arrow
    # Plot the line with arrows
    ax.annotate('', xy=(arrow_x, arrow_y_end), xytext=(arrow_x, arrow_y_start),
                 arrowprops=dict(arrowstyle='<->', color='k', lw=1.5), annotation_clip=False)
    # Add text annotation next to the arrow
    ax.text(arrow_x + 3, (arrow_y_start + arrow_y_end) / 2 - 0.05, arrow_text, color='k', fontsize=12)

    ax.tick_params(axis='x', labelsize=tick_label_size)
    ax.tick_params(axis='y', labelsize=tick_label_size)
    ax.set_ylim(-0.2,max_y)
    ax.set_xlim(0,250)
    ax.set_xlabel("HP Production, TWh/year", fontsize=label_font_size)
    ax.set_ylabel("Total Emissions, MtCO$_{2e}$ / annum", fontsize=label_font_size)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1,figsize=(8, 10))
plt.subplots_adjust(hspace=0.3)
make_ghg_intensity_figure(
    ax1, num_objectives=exec_options[option]['nobj'], max_y = exec_options[option]['max_y_int'])
make_ghg_emissions_figure(
    ax2, num_objectives=exec_options[option]['nobj'], max_y = exec_options[option]['max_y_em'])

In [ ]:
fig_options = ['png']
fig_folder = pathlib.Path("outputs/figures/moo")
if 'png' in fig_options:  
    fig.savefig(fig_folder / f'{option}_moo_results_fig.png', bbox_inches='tight', dpi=600, transparent=True)
if 'pdf' in fig_options:
    fig.savefig(fig_folder / f'{option}_moo_results_fig.pdf', bbox_inches='tight')
if 'svg' in fig_options:
    fig.savefig(fig_folder / f'{option}_moo_results_fig.svg', bbox_inches='tight')

In [ ]:
# Construct a dictionary with scenarios as keys and sets of constructed dams as values
def map_ids_to_ifc(scenario_ids: Dict[str, Set[int]], id_map = id_map) -> Dict[str, List[int]]:
    """ """
    return {key : list(set_remap(optim_ids, id_map)) for key, optim_ids in scenario_ids.items()}

sc_dams: Dict[str, Set[int]] = map_ids_to_ifc({
    "Ib": built_min['Dam IDs'],
    "Inb": notbuilt_current['Dam IDs'],
    "IIb": built_100['Dam IDs'],
    "IInb": notbuilt_100['Dam IDs'],
    "IIIb": built_200['Dam IDs'],
    "IIInb": notbuilt_200['Dam IDs']
})
with open(pathlib.Path(f'intermediate/optim_scenarios_{option}.json'), 'w') as file:  
    json_string = json.dumps(sc_dams, indent=4)
    file.write(json_string)

### Plot parallel plot of solutions

In [ ]:
# Only works for 5 objectives
merged_df_vis = merged_df.copy()
merged_df_vis['GHG emissions<br>[Mt CO2,eq/year]'] = \
    merged_df_vis['GHG emissions [tonne CO<sub>2,eq</sub>/year]']/1000_000
merged_df_vis = merged_df_vis.rename(columns={
    'Agricultural land loss, [km<sup>2</sup>]' : 'Agricultural land<br>loss, [km2]',
    'Deforestation, [km<sup>2</sup>]' : 'Deforestation<br>[km2]',
    'Land loss, [km<sup>2</sup>]' : 'Land loss<br>[km2]',
    'GHG intensity [gCO<sub>2,eq</sub>/kWh]' : 'Biogenic GHG intensity<br>[gCO2,eq/kWh]',
    'Mean HP [GWh/d]' : 'Mean HP<br>[GWh/d]',
    'Firm HP [GWh/d]' : 'Firm HP<br>[GWh/d]'
})
merged_df_vis['Scenario, [1/0]'] = merged_df_vis['Scenario, [1/0]'].astype('float')

mya_vis = OutputVisualiser(merged_df_vis) #merged_df)
fig = mya_vis.plot_parallel(
    columns = (
        'Mean HP<br>[GWh/d]', 
        'Firm HP<br>[GWh/d]', 
        'GHG emissions<br>[Mt CO2,eq/year]',
        'Agricultural land<br>loss, [km2]', 
        'Deforestation<br>[km2]', 
        'Land loss<br>[km2]', 
        'Biogenic GHG intensity<br>[gCO2,eq/kWh]',
        'Firm Power Ratio, [%]',
        'Scenario, [1/0]'), 
    labels = {
        'Mean HP<br>[GWh/d]' : 'Mean HP',
        'Firm HP<br>[GWh/d]' : 'Firm HP',
        'GHG emissions<br>[Mt CO2,eq/year]': 'GHG emissions',
        'Agricultural land<br>loss, [km2]' : 'Agricultural land loss',
        'Deforestation<br>[km2]' : 'Deforestation',
        'Land loss<br>[km2]' : 'Land loss',
        'Biogenic GHG intensity<br>[gCO2,eq/kWh]' : 'GHG intensity',
        'Firm Power Ratio, [%]' : 'Firm power ratio',
        'Scenario, [1/0]' : 'Scenario, [1/0]'
    },
    color_col = 'Biogenic GHG intensity<br>[gCO2,eq/kWh]', color_limits=(0,60))
plotly_jinja_data = {"fig":fig.to_html(full_html=False)}
#consider also defining the include_plotlyjs parameter to point to an external Plotly.js as described above

output_fig_folder = pathlib.Path("outputs/figures/moo")
output_fig_folder.mkdir(parents=True, exist_ok=True)
output_html_path=output_fig_folder / f"parallel_plot_{option}.html"
input_template_path = pathlib.Path("templates/parallel_plot_template.html")
with open(output_html_path, "w", encoding="utf-8") as output_file:
    with open(input_template_path) as template_file:
        j2_template = Template(template_file.read())
        output_file.write(j2_template.render(plotly_jinja_data))

## The End